<a href="https://colab.research.google.com/github/ashispapu/LLMs/blob/main/Prompt_Gemma_like_a_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://res.infoq.com/news/2024/02/google-gemma-open-model/en/headerimage/generatedHeaderImage-1708977571481.jpg" width="400"></center>

# Introduction

This Notebook is just for me to check a bit Gemma model.
Will just load Gemma and demonstrate some very simple use cases.
I intend to learn from this experience so that I can then build someting a bit more complex.

# Install Keras NLP and Keras

In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

# Import packages

In [ ]:
import keras
import keras_nlp
import os

# Select the desired backend for Keras. Options: "jax", "tensorflow", or "torch".
os.environ["KERAS_BACKEND"] = "jax"  # Adjust as needed.

# Specific to the JAX backend, this setting helps avoid memory fragmentation, ensuring more efficient resource use.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

# Load Gemma Causal LLM through Keras NLP

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

# Check Gemma parameters

In [ ]:
gemma_lm.summary()

# Define and run a simple prompt (geography question)

In [ ]:
prompt = """
You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.
Question: {question}
Answer:
"""
response = gemma_lm.generate(prompt.format(question="What is the surface temperature of the Moon?"), max_length=256)
print(response)

# Format the output

In [ ]:
from IPython.display import display, Markdown

def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer"], ["blue", "red", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [ ]:
response = gemma_lm.generate(prompt.format(question="What is the surface temperature of the Moon?"), max_length=256)
display(Markdown(colorize_text(response)))

# Try another one (math problem)

In [ ]:
prompt = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""
response = gemma_lm.generate(prompt.format(question="Please write a function in Python to calculate the area of a circle of radius r"), max_length=256)
display(Markdown(colorize_text(response)))

# Multiple parameters questions

I create here a prompt with multiple parameters and I ask a variety of questions.

In [ ]:
prompt = """
You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
Question: What are the best {number} {items} from {place}?
Answer:
"""
response = gemma_lm.generate(prompt.format(
    number="3",
    items="food",
    place="France"
    ),
    max_length=256)
display(Markdown(colorize_text(response)))

In [ ]:
response = gemma_lm.generate(prompt.format(
    number="five",
    items="attractions",
    place="Italy"
    ),
    max_length=256)
display(Markdown(colorize_text(response)))

In [ ]:
response = gemma_lm.generate(prompt.format(
    number="two",
    items="places to retire",
    place="Spain"
    ),
    max_length=256)
display(Markdown(colorize_text(response)))

# Reasoning like Einstein will do

In [ ]:
prompt = """
You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h.
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti.
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.
Reasoning: Think step by step. Explain your reasoning.
Question: {question}
Answer:
"""
response = gemma_lm.generate(prompt.format(question="How many kilometers will travel totally the swallow?"),
    max_length=512)
display(Markdown(colorize_text(response)))

# Conclusions


## Using gemma_2b_en

Initially, I used the model `gemma_2b_en` and I was not really happy with the results. The main problems I found were:
* The model will answer incorectly to the question
* The model will answer to more question than the one answered
* The model will stop suddently in the middle of a phrase

## Using gemma_instruct_2b_en

Then, I got a good advice in the comments from one of the Kagglers (thank you @paultimothymooney) and switched to `gemma_instruct_2b_en` with much better results.

Here is the analysis of results with `gemma_instruct_2b_en`:
* General question: answer to the point, and the answer seems sensible. Well, it is actually not entirely correct, but it is close. And the question did not clarify if it is during solar exposure or in the shadow.
* Math problem, with answer as Python code: correct, and code is documented.
* Multi-parameters question: all answers are correct.
* More difficult math problem, requiring reasoning: the answer is not correct, because the reasoning is not correct but, to be fair, ... the problem contains a paradox. Did you (intelligent human reading this) notice what is the paradox?
